In [1]:
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob as glob
import os
import pandas as pd
from skimage import io, filters, morphology
import cv2
from scipy import ndimage as ndi
from pathlib import Path
%matplotlib inline
os.chdir('/Users/phoebewhite/Library/CloudStorage/GoogleDrive-phwhite@sas.upenn.edu/.shortcut-targets-by-id/12tKUWWdMW6Is2j_X0QWarfY8tIR_UEj9/White_Shared/cloud/experiment_ids/pw018/2023_10_12_cellarea_outputs')
Image.MAX_IMAGE_PIXELS = 933120000

In [2]:
Path(os.getcwd()+'/percent_confluence').mkdir(parents=True, exist_ok=True)


all wells

In [ ]:
# Get the tiffs to process
files = glob.glob('/Users/phoebewhite/Library/CloudStorage/GoogleDrive-phwhite@sas.upenn.edu/.shortcut-targets-by-id/12tKUWWdMW6Is2j_X0QWarfY8tIR_UEj9/White_Shared/cloud/experiment_ids/pw018/converted/used/*_5000_*.tif')
#files = glob.glob('/Users/phoebewhite/Library/CloudStorage/GoogleDrive-phwhite@sas.upenn.edu/.shortcut-targets-by-id/12tKUWWdMW6Is2j_X0QWarfY8tIR_UEj9/White_Shared/cloud/experiment_ids/pw003/converted/20220509_093117_811__Plate2022_05_09_pw003_naive_tgfb_Well1_ChannelDAPI_Seq0000.tif')

# build data fream for output
df_areas = pd.DataFrame(columns = ['Condition','Cell_area'])

# Maximum size filter for removing large objects that arent cells
max_size = 1000

for file in files:
    imname = file.split('/')[-1][:-4]
    print(imname)
    Path(os.getcwd()+'/percent_confluence/'+imname).mkdir(parents=True, exist_ok=True)

    image = io.imread(file)

    # Set up the figure
    #fig = plt.figure(figsize=(12, 12), dpi=300)

    SMALL_SIZE = 2
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=SMALL_SIZE)  # fontsize of the figure title


    # Plot original image
    #ax = fig.add_subplot(5, 2, 1)
    #imgplot = plt.imshow(image)
    #imgplot.set_clim(200, 1400)
    #ax.set_title('Original image')

    # Cut the image
    h,w = image.shape
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(400,400),(h-400,w-400)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    #ax = fig.add_subplot(5, 2, 2)
    cut_im = image.copy()
    cut_im[lum_im_arr == 0] = 0
    #imgplot = plt.imshow(cut_im)
    #imgplot.set_clim(200, 1400)
    #ax.set_title('Cut image')


    # Gaussian filter and then plot
    #ax = fig.add_subplot(5, 2, 3)
    image_gauss = filters.gaussian(cut_im, sigma=2)
    #imgplot = plt.imshow(image_gauss)
    #imgplot.set_clim(0.005, .02)
    #ax.set_title('Gaussian filtered')

    # Median filter the Gaussian filters and plot
    #ax = fig.add_subplot(5, 2, 4)
    image_gauss_median = filters.median(image_gauss)
    #imgplot = plt.imshow(image_gauss_median)
    #imgplot.set_clim(0.005, .02)
    #ax.set_title('Gaussian and median filtered')

    # Threshold image to create binary image
    #ax = fig.add_subplot(5, 2, 5)
    threshold = filters.threshold_niblack(image_gauss_median)
    binary_temp = image_gauss_median > threshold*1.01
    #imgplot = plt.imshow(binary_temp)
    #imgplot.set_clim(0, 1)
    #ax.set_title('Binary')

    # Perform morphological operations to fill small gaps and remove small objects
    #ax = fig.add_subplot(5, 2, 6)
    binary_temp = morphology.remove_small_objects(binary_temp, min_size=64)
    ## Remove the laerge segments as well
    binary = binary_temp.copy()
    selem = ndi.generate_binary_structure(binary.ndim, 1)
    ccs = np.zeros_like(binary_temp, dtype=np.int32)
    ndi.label(binary_temp, selem, output=ccs)
    cell_sizes = np.bincount(ccs.ravel())
    too_large = cell_sizes > max_size
    too_large_mask = too_large[ccs]
    binary[too_large_mask] = 0
    #imgplot = plt.imshow(binary)
    #imgplot.set_clim(0, 1)
    #ax.set_title('Binary gap filled')

    # remove border of false cells
    #ax = fig.add_subplot(5, 2, (7,10))
    binary_crop = binary.copy()
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(550,550),(h-550,w-550)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    binary_crop[lum_im_arr ==0] = 0
    #imgplot = plt.imshow(binary_crop)
    #imgplot.set_clim(0, 1)
    #ax.set_title('Binary gap filled and border removed')
    
    # Save the step by step image processing
    #plt.savefig('percent_confluence/' + imname + '/' + imname + '_adjusting_steps.png')
    
    # Save the binarized image
    plt.imsave('percent_confluence/' + imname + '/' + imname +  '_binarized.tiff', binary_crop)
    
    # Add the area to the output data frame
    df_areas.loc[len(df_areas)] = np.append(np.asarray(imname), np.sum(binary_crop))

# Add in the area that was cropped for % confluence calculations
df_areas.loc[len(df_areas)] = np.append('Blank', np.sum(lum_im_arr))

df_areas.to_csv('cell_areas_5000.csv')


# Get the tiffs to process
files = glob.glob('/Users/phoebewhite/Library/CloudStorage/GoogleDrive-phwhite@sas.upenn.edu/.shortcut-targets-by-id/12tKUWWdMW6Is2j_X0QWarfY8tIR_UEj9/White_Shared/cloud/experiment_ids/pw018/converted/used/*_1000_*.tif')
#files = glob.glob('/Users/phoebewhite/Library/CloudStorage/GoogleDrive-phwhite@sas.upenn.edu/.shortcut-targets-by-id/12tKUWWdMW6Is2j_X0QWarfY8tIR_UEj9/White_Shared/cloud/experiment_ids/pw003/converted/20220509_093117_811__Plate2022_05_09_pw003_naive_tgfb_Well1_ChannelDAPI_Seq0000.tif')

# build data fream for output
df_areas = pd.DataFrame(columns = ['Condition','Cell_area'])

# Maximum size filter for removing large objects that arent cells
max_size = 1000

for file in files:
    imname = file.split('/')[-1][:-4]
    print(imname)
    Path(os.getcwd()+'/percent_confluence/'+imname).mkdir(parents=True, exist_ok=True)

    image = io.imread(file)

    # Set up the figure
    #fig = plt.figure(figsize=(12, 12), dpi=300)

    SMALL_SIZE = 2
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=SMALL_SIZE)  # fontsize of the figure title


    # Plot original image
    #ax = fig.add_subplot(5, 2, 1)
    #imgplot = plt.imshow(image)
    #imgplot.set_clim(200, 1400)
    #ax.set_title('Original image')

    # Cut the image
    h,w = image.shape
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(400,400),(h-400,w-400)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    #ax = fig.add_subplot(5, 2, 2)
    cut_im = image.copy()
    cut_im[lum_im_arr == 0] = 0
    #imgplot = plt.imshow(cut_im)
    #imgplot.set_clim(200, 1400)
    #ax.set_title('Cut image')


    # Gaussian filter and then plot
    #ax = fig.add_subplot(5, 2, 3)
    image_gauss = filters.gaussian(cut_im, sigma=2)
    #imgplot = plt.imshow(image_gauss)
    #imgplot.set_clim(0.005, .02)
    #ax.set_title('Gaussian filtered')

    # Median filter the Gaussian filters and plot
    #ax = fig.add_subplot(5, 2, 4)
    image_gauss_median = filters.median(image_gauss)
    #imgplot = plt.imshow(image_gauss_median)
    #imgplot.set_clim(0.005, .02)
    #ax.set_title('Gaussian and median filtered')

    # Threshold image to create binary image
    #ax = fig.add_subplot(5, 2, 5)
    threshold = filters.threshold_niblack(image_gauss_median)
    binary_temp = image_gauss_median > threshold*1.01
    #imgplot = plt.imshow(binary_temp)
    #imgplot.set_clim(0, 1)
    #ax.set_title('Binary')

    # Perform morphological operations to fill small gaps and remove small objects
    #ax = fig.add_subplot(5, 2, 6)
    binary_temp = morphology.remove_small_objects(binary_temp, min_size=64)
    ## Remove the laerge segments as well
    binary = binary_temp.copy()
    selem = ndi.generate_binary_structure(binary.ndim, 1)
    ccs = np.zeros_like(binary_temp, dtype=np.int32)
    ndi.label(binary_temp, selem, output=ccs)
    cell_sizes = np.bincount(ccs.ravel())
    too_large = cell_sizes > max_size
    too_large_mask = too_large[ccs]
    binary[too_large_mask] = 0
    #imgplot = plt.imshow(binary)
    #imgplot.set_clim(0, 1)
    #ax.set_title('Binary gap filled')

    # remove border of false cells
    #ax = fig.add_subplot(5, 2, (7,10))
    binary_crop = binary.copy()
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(550,550),(h-550,w-550)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    binary_crop[lum_im_arr ==0] = 0
    #imgplot = plt.imshow(binary_crop)
    #imgplot.set_clim(0, 1)
    #ax.set_title('Binary gap filled and border removed')
    
    # Save the step by step image processing
    #plt.savefig('percent_confluence/' + imname + '/' + imname + '_adjusting_steps.png')
    
    # Save the binarized image
    plt.imsave('percent_confluence/' + imname + '/' + imname +  '_binarized.tiff', binary_crop)
    
    # Add the area to the output data frame
    df_areas.loc[len(df_areas)] = np.append(np.asarray(imname), np.sum(binary_crop))

# Add in the area that was cropped for % confluence calculations
df_areas.loc[len(df_areas)] = np.append('Blank', np.sum(lum_im_arr))

df_areas.to_csv('cell_areas_1000.csv')



# Get the tiffs to process
files = glob.glob('/Users/phoebewhite/Library/CloudStorage/GoogleDrive-phwhite@sas.upenn.edu/.shortcut-targets-by-id/12tKUWWdMW6Is2j_X0QWarfY8tIR_UEj9/White_Shared/cloud/experiment_ids/pw018/converted/used/*_500_*.tif')
#files = glob.glob('/Users/phoebewhite/Library/CloudStorage/GoogleDrive-phwhite@sas.upenn.edu/.shortcut-targets-by-id/12tKUWWdMW6Is2j_X0QWarfY8tIR_UEj9/White_Shared/cloud/experiment_ids/pw003/converted/20220509_093117_811__Plate2022_05_09_pw003_naive_tgfb_Well1_ChannelDAPI_Seq0000.tif')

# build data fream for output
df_areas = pd.DataFrame(columns = ['Condition','Cell_area'])

# Maximum size filter for removing large objects that arent cells
max_size = 1000

for file in files:
    imname = file.split('/')[-1][:-4]
    print(imname)
    Path(os.getcwd()+'/percent_confluence/'+imname).mkdir(parents=True, exist_ok=True)

    image = io.imread(file)

    # Set up the figure
    #fig = plt.figure(figsize=(12, 12), dpi=300)

    SMALL_SIZE = 2
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=SMALL_SIZE)  # fontsize of the figure title


    # Plot original image
    #ax = fig.add_subplot(5, 2, 1)
    #imgplot = plt.imshow(image)
    #imgplot.set_clim(200, 1400)
    #ax.set_title('Original image')

    # Cut the image
    h,w = image.shape
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(400,400),(h-400,w-400)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    #ax = fig.add_subplot(5, 2, 2)
    cut_im = image.copy()
    cut_im[lum_im_arr == 0] = 0
    #imgplot = plt.imshow(cut_im)
    #imgplot.set_clim(200, 1400)
    #ax.set_title('Cut image')


    # Gaussian filter and then plot
    #ax = fig.add_subplot(5, 2, 3)
    image_gauss = filters.gaussian(cut_im, sigma=2)
    #imgplot = plt.imshow(image_gauss)
    #imgplot.set_clim(0.005, .02)
    #ax.set_title('Gaussian filtered')

    # Median filter the Gaussian filters and plot
    #ax = fig.add_subplot(5, 2, 4)
    image_gauss_median = filters.median(image_gauss)
    #imgplot = plt.imshow(image_gauss_median)
    #imgplot.set_clim(0.005, .02)
    #ax.set_title('Gaussian and median filtered')

    # Threshold image to create binary image
    #ax = fig.add_subplot(5, 2, 5)
    threshold = filters.threshold_niblack(image_gauss_median)
    binary_temp = image_gauss_median > threshold*1.01
    #imgplot = plt.imshow(binary_temp)
    #imgplot.set_clim(0, 1)
    #ax.set_title('Binary')

    # Perform morphological operations to fill small gaps and remove small objects
    #ax = fig.add_subplot(5, 2, 6)
    binary_temp = morphology.remove_small_objects(binary_temp, min_size=64)
    ## Remove the laerge segments as well
    binary = binary_temp.copy()
    selem = ndi.generate_binary_structure(binary.ndim, 1)
    ccs = np.zeros_like(binary_temp, dtype=np.int32)
    ndi.label(binary_temp, selem, output=ccs)
    cell_sizes = np.bincount(ccs.ravel())
    too_large = cell_sizes > max_size
    too_large_mask = too_large[ccs]
    binary[too_large_mask] = 0
    #imgplot = plt.imshow(binary)
    #imgplot.set_clim(0, 1)
    #ax.set_title('Binary gap filled')

    # remove border of false cells
    #ax = fig.add_subplot(5, 2, (7,10))
    binary_crop = binary.copy()
    lum_im = Image.new('L', [w,h],0)
    draw = ImageDraw.Draw(lum_im)
    draw.pieslice([(550,550),(h-550,w-550)],0,360,fill = 1)
    lum_im_arr = np.asarray(lum_im)
    binary_crop[lum_im_arr ==0] = 0
    #imgplot = plt.imshow(binary_crop)
    #imgplot.set_clim(0, 1)
    #ax.set_title('Binary gap filled and border removed')
    
    # Save the step by step image processing
    #plt.savefig('percent_confluence/' + imname + '/' + imname + '_adjusting_steps.png')
    
    # Save the binarized image
    plt.imsave('percent_confluence/' + imname + '/' + imname +  '_binarized.tiff', binary_crop)
    
    # Add the area to the output data frame
    df_areas.loc[len(df_areas)] = np.append(np.asarray(imname), np.sum(binary_crop))

# Add in the area that was cropped for % confluence calculations
df_areas.loc[len(df_areas)] = np.append('Blank', np.sum(lum_im_arr))

df_areas.to_csv('cell_areas_500.csv')


20230509_101804_247__Platepw018_high_5000_p1_Well2_ChannelDAPI_Seq0001
20230509_140110_057__Platepw018_high_5000_p2_Well2_ChannelDAPI_Seq0001
20230424_103823_519__Platepw018_low_5000_p2_Well3_ChannelDAPI_Seq0002
20230509_140110_057__Platepw018_high_5000_p2_Well6_ChannelDAPI_Seq0003
20230423_092759_784__Platepw018_low_5000_p1_Well4_ChannelDAPI_Seq0005
